In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
standings_urls = ["https://fbref.com/en/comps/9/Premier-League-Stats",
    "https://fbref.com/en/comps/11/Serie-A-Stats",
    "https://fbref.com/en/comps/12/La-Liga-Stats",
    "https://fbref.com/en/comps/13/Ligue-1-Stats",
    "https://fbref.com/en/comps/20/Bundesliga-Stats"]

leagues =  pd.read_excel('./../Club_name.xlsx')

In [13]:
def clean(table, league):
    if ( 'Unnamed: 4_level_0',       'MP') in table.columns:    
        table = table[[( 'Unnamed: 0_level_0',   'Player'),
            ( 'Unnamed: 1_level_0',   'Nation'),
            ( 'Unnamed: 2_level_0',      'Pos'),
            ( 'Unnamed: 3_level_0',      'Age'),
            ( 'Unnamed: 4_level_0',       'MP'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast')]][:-2]
    else:
        table = table[[( 'Unnamed: 0_level_0',   'Player'),
            ( 'Unnamed: 1_level_0',   'Nation'),
            ( 'Unnamed: 2_level_0',      'Pos'),
            ( 'Unnamed: 3_level_0',      'Age'),
            ( 'Playing Time',       'MP'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast')]][:-2]

    table.columns = table.columns.droplevel()
    table.insert(1, 'club_name', league, True)
    table['Gls'] = table['Gls'].fillna(0).astype(int)
    table['Ast'] = table['Ast'].fillna(0).astype(int)
    table['Nation'] = [m[-3:] if (type(m) == str and len(m) >= 3) else None for m in table['Nation']]
    table['Age'] = [m[:2] if (type(m) == str and len(m) > 0) else None for m in table['Age']]
    return table

In [21]:
player_df = pd.DataFrame(columns=['Player','club_name', "Nation", 'Pos', 'Age', "MP", 'Gls', 'Ast'])
count = 19

In [33]:
player_df = pd.read_csv('./playerDB591.csv')
player_df.head()

,Player,club_name,Nation,Pos,Age,MP,Gls,Ast
0,Gabriel Dos Santos,Arsenal,BRA,DF,25.0,20,2,0
1,Aaron Ramsdale,Arsenal,ENG,GK,24.0,20,0,0
2,William Saliba,Arsenal,FRA,DF,21.0,20,2,1
3,Bukayo Saka,Arsenal,ENG,FW,21.0,20,7,7
4,Granit Xhaka,Arsenal,SUI,MF,30.0,20,3,5


In [34]:
count = 79
for standings_url, league in zip(standings_urls[3:],leagues[['Ligue 1', 'Bundesliga']]):
    data = requests.get(standings_url) 
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l for l in links if '/squads/' in l.get('href')]
    team_list = [l.get_text() for l in links]
    links = [l.get('href') for l in links]
    standing = pd.read_html(data.text, attrs={'class': 'stats_table'})[0]
    team_urls = [f"https://fbref.com{l}" for l in links]

    for i, (l, team_name) in enumerate((zip(team_urls, team_list))):
        if i == len(team_urls): 
            break     
        d = requests.get(l)
        match = clean(pd.read_html(d.text, match = "Standard Stats")[0], team_name)
        player_df = pd.concat([player_df, match])
        print(count)
        count += 1
    player_df.to_csv(f'./playerDB{count}1.csv', index=False)

79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [23]:
count

32

In [35]:
player_df.describe()

,MP,Gls,Ast
count,3273.000000,3273.000000,3273.000000
mean,9.478766,0.823404,0.577757
std,7.551105,1.833198,1.181363
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,10.000000,0.000000,0.000000
75%,17.000000,1.000000,1.000000
max,22.000000,25.000000,11.000000


In [40]:
player_info_df = player_df[['Player', 'Nation', 'Pos', 'Age']]
player_info_df.insert(0, 'player_id', list(range(1, len(player_info_df.index) + 1)), False)

In [71]:
player_info_df['Age'] = player_info_df['Age'].fillna(0).astype(int)
player_info_df.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_21656\3710168148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info_df['Age'] = player_info_df['Age'].fillna(0).astype(int)


,player_id,Player,Nation,Pos,Age
0,1,Gabriel Dos Santos,BRA,DF,25
1,2,Aaron Ramsdale,ENG,GK,24
2,3,William Saliba,FRA,DF,21
3,4,Bukayo Saka,ENG,FW,21
4,5,Granit Xhaka,SUI,MF,30


In [72]:
player_info_df.to_csv('./player_infoDB.csv', header=False, index=False)

In [73]:
is_member_df = player_df[['club_name', 'MP', 'Gls', 'Ast']]
is_member_df.head()

,club_name,MP,Gls,Ast
0,Arsenal,20,2,0
1,Arsenal,20,0,0
2,Arsenal,20,2,1
3,Arsenal,20,7,7
4,Arsenal,20,3,5


In [74]:
is_member_df.insert(0, 'player_id', list(range(1, len(is_member_df.index) + 1)), False)
is_member_df.insert(2, 'season', '2022-2023', True)
is_member_df.head()

,player_id,club_name,season,MP,Gls,Ast
0,1,Arsenal,2022-2023,20,2,0
1,2,Arsenal,2022-2023,20,0,0
2,3,Arsenal,2022-2023,20,2,1
3,4,Arsenal,2022-2023,20,7,7
4,5,Arsenal,2022-2023,20,3,5


In [42]:
club_df = pd.read_csv('./../club/clubDB.csv')
club_df.head()

,club_id,club_name,nation,stadium
0,1,Manchester Utd,ENG,1
1,2,West Ham,ENG,2
2,3,Tottenham,ENG,3
3,4,Arsenal,ENG,4
4,5,Manchester City,ENG,5


In [52]:
team = list(club_df['club_name'])
id = list(club_df['club_id'])

In [59]:
print(team)
print(type(id[0]))

['Manchester Utd', 'West Ham', 'Tottenham', 'Arsenal', 'Manchester City', 'Liverpool', 'Newcastle Utd', 'Aston Villa', 'Chelsea', 'Everton', 'Leeds United', 'Southampton', 'Leicester City', 'Wolves', 'Brighton', "Nott'ham Forest", 'Crystal Palace', 'Fulham', 'Brentford', 'Bournemouth', 'Milan', 'Inter', 'Roma', 'Lazio', 'Napoli', 'Fiorentina', 'Juventus', 'Hellas Verona', 'Bologna', 'Sampdoria', 'Lecce', 'Salernitana', 'Torino', 'Udinese', 'Sassuolo', 'Empoli', 'Atalanta', 'Cremonese', 'Monza', 'Spezia', 'Barcelona', 'Real Madrid', 'Atlético Madrid', 'Betis', 'Athletic Club', 'Valencia', 'Sevilla', 'Espanyol', 'Real Sociedad', 'Elche', 'Celta Vigo', 'Valladolid', 'Osasuna', 'Villarreal', 'Cádiz', 'Mallorca', 'Getafe', 'Almería', 'Rayo Vallecano', 'Girona', 'Marseille', 'Lyon', 'Lille', 'Paris S-G', 'Lens', 'Nantes', 'Nice', 'Toulouse', 'Montpellier', 'Rennes', 'Strasbourg', 'Troyes', 'Reims', 'Angers', 'Lorient', 'Auxerre', 'Monaco', 'Brest', 'Clermont Foot', 'Ajaccio', 'Dortmund', 'Ba

In [75]:
temp = is_member_df
temp['club_name'] = temp['club_name'].replace(team, id)
temp.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_21656\1755521727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['club_name'] = temp['club_name'].replace(team, id)


,player_id,club_name,season,MP,Gls,Ast
0,1,4,2022-2023,20,2,0
1,2,4,2022-2023,20,0,0
2,3,4,2022-2023,20,2,1
3,4,4,2022-2023,20,7,7
4,5,4,2022-2023,20,3,5


In [76]:
is_member_df = temp
is_member_df.head()

,player_id,club_name,season,MP,Gls,Ast
0,1,4,2022-2023,20,2,0
1,2,4,2022-2023,20,0,0
2,3,4,2022-2023,20,2,1
3,4,4,2022-2023,20,7,7
4,5,4,2022-2023,20,3,5


In [77]:
is_member_df.to_csv('./is_memberDB.csv', index=False, header=False)